In [19]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers,optimizers,losses,metrics,models

from json import load

In [20]:
with open("../data/example_build.json","r") as build_file:
    build_config = load(build_file)
    
with open("../data/layer_config.json","r") as config_file:
    layer_config = load(config_file)

In [21]:
layers = {}

for l in layer_config.values():
    layers.update({ dl['name']:dl for dl in l['layers'] })

In [22]:
inputs = []

for _id,config in build_config.items():
    if config['name'] == 'Input':
        inputs.append(_id)

In [23]:
levels = [ set() for i in range(len(build_config))]

def h(node,config,di=0):
    print (di,node,build_config[node]['name'])
    levels[di].add(node)
    if build_config[node]['connections']['outbound']:
        for next_node in build_config[node]['connections']['outbound']: 
            h(next_node,build_config,di+1)
    
h(inputs[0],build_config,0)

0 layer_0 Input
1 layer_1 Conv2D
2 layer_2 Conv2D
3 layer_4 Conv2D
4 layer_5 Model
5 layer_6 Compile
6 layer_7 Train
2 layer_3 Conv2D
3 layer_4 Conv2D
4 layer_5 Model
5 layer_6 Compile
6 layer_7 Train
2 layer_18 Conv2D
3 layer_4 Conv2D
4 layer_5 Model
5 layer_6 Compile
6 layer_7 Train
2 layer_19 Conv2D
3 layer_4 Conv2D
4 layer_5 Model
5 layer_6 Compile
6 layer_7 Train


In [27]:
levels = [list(level) for level in levels if len(level)]
levels

[['layer_0'],
 ['layer_1'],
 ['layer_18', 'layer_2', 'layer_19', 'layer_3'],
 ['layer_4'],
 ['layer_5'],
 ['layer_6'],
 ['layer_7']]

In [34]:
model_config = None
compile_config = None
train_config = None

for layer in build_config:
    print (build_config[layer]['name'])

Input
Conv2D
Conv2D
Conv2D
Conv2D
Model
Compile
Train
Conv2D
Conv2D
Conv2D


In [13]:
# def create_layer(layer_id,layer_config):
#     arguments = ",\n".join([ 
#         f'\t{arg}={val}' for arg,val in layer_config['arguments'].items()  
#     ])
    
#     input_layers = layer_config['connections']['inbound']
#     input_layers = '[' + ', '.join(input_layers) + ']' if len(input_layers) > 1 else ( input_layers[0] if len(input_layers) else None )
    
#     return f"""{layer} = layer.{layer_config['name']}(
#     {arguments}
#     ){ '(' + input_layers + ')' if input_layers else ''}\n\n"""

# build = ""

# for level in levels:
#     for layer in level:
#         build += create_layer(layer,build_config[layer])

In [14]:
# model = f"keras.Model([{ ', '.join(inputs) },], [{ ', '.join(levels[-1]) },])"

In [15]:
# code = """
# import tensorflow as tf

# from tensorflow import keras
# from tensorflow.keras import layers,optimizers,losses,metrics,models

# {build}

# {model}
# """

In [17]:
# print (code.format(build=build,model=model))